##Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cd /content/drive/My\ Drive/Data_mining/Lab2/Notebooks
# !ls

/content/drive/My Drive/Data_mining/Lab2/Notebooks
Decision_Tree.ipynb  Load_Dataset.ipynb  Naive_Bayes.ipynb  RunAndPlot.ipynb


#Installing Packages

In [ ]:
# !pip install ipynb

##Importing Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import math
import csv
import time
import re
import tracemalloc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
# from ipynb.fs.full.Load_Dataset import load_dataset, get_dataset_names

plt.rc('xtick',labelsize=15)
plt.rc('ytick',labelsize=15)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Naive Bayes Implementation


In [ ]:
randomState = 123

In [ ]:
def gaussianDist(x, mu, sigma):
  return (np.exp(-((x-mu)**2)/(2*(sigma**2))))/(np.sqrt(2*np.pi)*sigma)

  
def predict(X, Y, xTest, cache):
  attrInfo = cache['attrInfo']
  attrMeans = cache['attrMeans']
  attrStds = cache['attrStds']
  classes = np.unique(Y)
  mxProb = -np.inf
  classLabel = None
  
  # print('attrMeans= ' , attrMeans)

  for cls in classes:
    projX, projY = X[Y==cls], Y[Y==cls]
    likeliHood = 0
    for i in range(len(xTest)):
      attrInfoDict = attrInfo[i]
      if attrInfoDict['type']=='discrete':
        if np.sum(projX[:,i]==xTest[i]) == 0:
          likeliHood += np.log(1e-6)
        else:
          condProb = np.sum(projX[:,i]==xTest[i])/projY.shape[0]
          if condProb == 0:
            condProb = 1e-6  
          likeliHood += np.log(condProb)
      else:
        stdEps = 0
        if attrStds[cls][i]==0:
          stdEps = 1e-6
        condProb = gaussianDist(xTest[i], attrMeans[cls][i], attrStds[cls][i]+stdEps)
        if condProb == 0:
          condProb = 1e-6
        likeliHood += np.log(condProb)
    classCondProb = likeliHood + np.log(projY.shape[0]/Y.shape[0])
    if classCondProb>mxProb:
      classLabel = cls
      mxProb = classCondProb
  return classLabel

def predictDataNaiveBayes(X, Y, XTest, cache):
  classes = np.unique(Y)        
  YPred = np.array([])
  for x in XTest:
    yPred = predict(X, Y, x, cache)
    YPred = np.append(YPred, yPred)
  return YPred

### Loading Dataset

In [ ]:
# # datasetName = 'sampleDataset2'
# datasetName = 'digits'
# # datasetName = 'australian'

# X, Y =   load_dataset(datasetName)
# threshold = 15
# attrMeans = {}
# attrStds = {}
# classes= np.unique(Y)
# attrInfo = []

# for i in range(X.shape[1]):
#   attrInfoDict = {}
#   values = np.unique(X[:,i])
#   if values.shape[0]>threshold:
#     attrInfoDict['type']= 'continuous'
#     attrInfoDict['values']= None
#   else:
#     attrInfoDict['type']= 'discrete'
#     attrInfoDict['values']= values
#   attrInfo.append(attrInfoDict) 

# for i in range(classes.shape[0]):
#   cls = classes[i]
#   projX, projY = X[Y==cls], Y[Y==cls]
#   attrMeans[cls], attrStds[cls] = [], []
#   for j in range(X.shape[1]):
#     attrInfoDict = attrInfo[j]
#     if attrInfoDict['type']=='continuous': 
#       attrMeans[cls].append(np.mean(projX[:,j]))
#       attrStds[cls].append(np.std(projX[:,j]))
#     else:
#       attrMeans[cls].append(None)
#       attrStds[cls].append(None)
# cache={
#     'attrInfo': attrInfo,
#     'attrMeans': attrMeans,
#     'attrStds': attrStds
#   }

# # print(X, Y)
# # print(trainSize, testSize)
# # print(attrInfo)
# # print(cache)
# print(attrMeans)
# print(attrStds)



dataset: digits
instances = 1797, features= 64 
{0: [None, None, 4.185393258426966, 13.095505617977528, 11.297752808988765, 2.9269662921348316, 0.033707865168539325, 0.0, None, 0.8876404494382022, 12.57865168539326, 13.370786516853933, 11.49438202247191, 11.376404494382022, 0.9662921348314607, 0.0, None, 3.730337078651685, 14.275280898876405, 5.264044943820225, 2.101123595505618, 12.168539325842696, 3.5224719101123596, None, None, 5.292134831460674, 12.713483146067416, 1.9943820224719102, 0.1404494382022472, 9.061797752808989, 6.449438202247191, None, None, 5.870786516853933, 11.5561797752809, 0.8932584269662921, 0.0449438202247191, 8.775280898876405, 7.123595505617978, None, None, 3.49438202247191, 13.286516853932584, 1.651685393258427, 1.5337078651685394, 11.308988764044944, 5.853932584269663, None, None, 0.797752808988764, 13.0561797752809, 9.960674157303371, 10.348314606741573, 13.252808988764045, 2.4157303370786516, 0.0, None, None, 4.157303370786517, 13.561797752808989, 13.325842

### Performance

In [ ]:
# # print(XTest)
# YPred = predictDataNaiveBayes(X, Y, X , cache)
# print(Y)
# print(YPred)
# precision, recall, f1score, _ = precision_recall_fscore_support(Y, YPred)
# print(np.sum(Y==YPred))
# print([i for i in range(Y.shape[0]) if Y[i]!=YPred[i]])
# accuracy = accuracy_score(Y, YPred)
# print('precision = {}, recall= {}, f1score={}, accuracy = {}'.format(precision, recall, f1score, accuracy))

[0 1 2 ... 8 9 8]
[0. 1. 8. ... 8. 9. 8.]
1542
[2, 5, 9, 18, 19, 23, 27, 31, 37, 39, 46, 50, 51, 54, 57, 59, 63, 69, 75, 77, 92, 96, 99, 103, 115, 116, 120, 125, 127, 129, 133, 161, 172, 175, 177, 215, 226, 245, 254, 265, 271, 275, 293, 325, 329, 341, 348, 354, 355, 361, 363, 375, 384, 385, 420, 446, 448, 449, 477, 492, 519, 527, 528, 529, 530, 537, 539, 547, 553, 555, 561, 575, 578, 579, 584, 589, 599, 604, 605, 607, 614, 621, 633, 639, 640, 643, 647, 665, 677, 678, 683, 685, 691, 700, 719, 731, 736, 737, 738, 739, 745, 746, 751, 753, 757, 770, 774, 779, 781, 784, 792, 794, 795, 808, 809, 811, 813, 818, 821, 823, 832, 835, 836, 838, 839, 847, 854, 856, 860, 861, 866, 868, 874, 879, 891, 899, 905, 906, 916, 950, 951, 985, 987, 1022, 1037, 1048, 1058, 1060, 1064, 1066, 1068, 1071, 1080, 1083, 1095, 1096, 1100, 1107, 1117, 1118, 1132, 1146, 1152, 1155, 1185, 1202, 1210, 1213, 1217, 1233, 1237, 1242, 1247, 1253, 1256, 1264, 1271, 1279, 1284, 1315, 1326, 1346, 1361, 1364, 1376, 1377, 1383,